### This Notebooks contains lexical exploration

In [27]:
!pip install emoji
!pip install emosent-py
!pip install dask


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


#### Loading Libs

In [28]:
import sys
sys.path.append("../../libs/leia/")
from leia import SentimentIntensityAnalyzer
import pandas as pd
import dask.dataframe as dd
import numpy as np
import emoji
from emosent import get_emoji_sentiment_rank
from datetime import datetime

#### Loading Classifiers

In [29]:
leia_classifier = SentimentIntensityAnalyzer()

#### Custom functions

In [30]:
def remove_duplicate_emoji(orig_str):
    prev_emoji = None
    remove_duplicate_emoji = []
    try:
        for c in orig_str:
            if c in emoji.EMOJI_DATA.keys():
                if prev_emoji == c:
                    continue
                prev_emoji = c
            remove_duplicate_emoji.append(c)
        return "".join(remove_duplicate_emoji)
    except:
        return orig_str

def check_emoji_sentiment(orig_str):
    neg = 0
    neu = 0
    pos = 0
    sco = 0
    
    try:
        emoji_list = emoji.distinct_emoji_list(orig_str)
        if len(emoji_list) > 0:
            for item in emoji_list:
                
                try:
                    sentiment_rank = get_emoji_sentiment_rank(item)
                    neg += sentiment_rank['negative']
                    neu += sentiment_rank['neutral']
                    pos += sentiment_rank['positive']
                    sco += sentiment_rank['sentiment_score']
                
                except:
                    pass
        
        tot = neg + neu + pos 
        
        return {
            'neg': round(neg/tot,3),
            'neu': round(neu/tot,3),
            'pos': round(pos/tot,3),
            'sco': round(sco/tot,3)
        }
    
    except:
        return {
            'neg': round(neg,3),
            'neu': round(neu,3),
            'pos': round(pos,3),
            'sco': round(sco,3)
        }

def check_text_sentiment(orig_str):
    orig_str_without_emoji =  emoji.replace_emoji(remove_duplicate_emoji(orig_str), replace='')
    sentiment_polarity = leia_classifier.polarity_scores(orig_str_without_emoji)
    
    return {
            'neg': sentiment_polarity['neg'],
            'neu': sentiment_polarity['neu'],
            'pos': sentiment_polarity['pos'],
            'compound': sentiment_polarity['compound'],
    }

def compound_sentiment(text_sentiment, emoji_sentiment):
    compound = {
        'neg': 0,
        'neu': 0,
        'pos': 0
    }
    tot = 0
    for kind in compound.keys():
        compound[kind] = text_sentiment[kind] + emoji_sentiment[kind]
        tot += compound[kind]
    return {k: round(v / tot,3) for k, v in compound.items()}

In [31]:
import uuid

d_type = {
    'text': np.str,
    'author_id': np.float64,
    'conversation_id': np.float64,
    'created_at': np.str,
    'source': np.str,
    'tweet_id': np.float64,
    'geo_id': np.str,
    'retweet_count': np.float64,
    'reply_count': np.float64,
    'like_count': np.float64,
    'quote_count': np.float64,
    'in_reply_to_user_id': np.float64,
    'referenced_type': np.str,
    'referenced_id': np.float64
}

# df = dd.read_csv('../../src/application/data/tweets_brazil.csv', dtype=d_type, error_bad_lines=False)  

# df_tweets = pd.read_csv('../../src/application/data/tweets_portuguese.csv', sep=',', dtype=d_type, error_bad_lines=False, low_memory=False)

chunksize = 10000
for chunk in pd.read_csv('../../src/application/data/tweets_portuguese.csv', sep=',', dtype=d_type, error_bad_lines=False, chunksize=chunksize):
    try:
        chunk=chunk[['text', 'created_at']]
        chunk['text'] = chunk['text'].apply(lambda x: remove_duplicate_emoji(x))
        chunk['text_sentiment'] = chunk['text'].apply(lambda x: check_text_sentiment(str(x)))
        chunk['sentiment_value'] = chunk.apply(lambda x: x['text_sentiment']['compound'], axis=1)
        chunk=chunk[['text', 'created_at', 'text_sentiment', 'sentiment_value']]
        chunk.to_csv(f'./data/{uuid.uuid4().hex}.csv', index=False, header=True)
    except ParserError:
        print(1)
        pass